You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [10]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [1]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [30]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [32]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [34]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [36]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [38]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [40]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [42]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [43]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

2024-05-30 18:16:00,842 - 30608 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [44]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan: 
Topic: Artificial Intelligence

Outline:
I. Introduction
- Definition of Artificial Intelligence
- Brief history and evolution of AI
- Importance and impact of AI in various industries

II. Latest Trends in Artificial Intelligence
- Machine learning and deep learning advancements
- Natural language processing and chatbots
- Autonomous vehicles and robotics
- AI in healthcare and personalized medicine
- Ethical considerations in AI development

III. Key Players in Artificial Intelligence
- Lead

- Display the results of your execution as markdown in the notebook.

In [45]:
from IPython.display import Markdown
Markdown(result)

# Artificial Intelligence: Revolutionizing Industries and Shaping the Future

## Introduction

Artificial Intelligence (AI) has become a prominent topic in today's technological landscape, with its roots dating back to the 1950s. AI involves the simulation of human intelligence processes by machines, particularly computer systems, and has evolved significantly over the years. The impact of AI is profound, with the potential to revolutionize various industries and change the way we live and work.

## Latest Trends in Artificial Intelligence

A key trend in AI is the rapid development of machine learning and deep learning algorithms, enabling computers to learn from data and make intelligent decisions autonomously. Natural language processing and chatbots have also gained popularity, transforming customer service and communication channels. Furthermore, advancements in autonomous vehicles and robotics promise safer and more efficient transportation systems. In healthcare, AI is being leveraged to personalize medicine and enhance patient outcomes, highlighting the diverse applications of AI across industries. However, ethical considerations in AI development and deployment are becoming increasingly crucial as the technology progresses.

## Key Players in Artificial Intelligence

Leading companies like Google, IBM, and Amazon are driving AI research and development, pushing the boundaries of innovation. Influential figures such as Andrew Ng and Yoshua Bengio have significantly contributed to shaping the future of AI. Additionally, emerging startups and innovations are bringing fresh perspectives and ideas to the AI landscape, fostering a dynamic and diverse environment for technological advancements.

## Noteworthy News in Artificial Intelligence

Recent breakthroughs in AI technology have garnered global attention, showcasing the vast potential of AI in solving complex real-world problems and enhancing daily experiences. Regulatory updates and policy developments play a critical role in shaping the future of AI, ensuring ethical standards are upheld and societal impacts are carefully considered.

## Conclusion

Artificial Intelligence stands as a transformative force, reshaping industries and fostering innovation on a global scale. To navigate this dynamic technological landscape effectively, it is crucial to stay informed about the latest trends and developments in AI. By engaging with reputable sources and industry experts, we can embrace the opportunities presented by AI while remaining mindful of the ethical implications and societal impacts of AI technologies.

**Call to Action:** Stay informed about the latest trends and developments in Artificial Intelligence by following reputable sources and engaging in discussions with industry experts. Embrace the opportunities AI presents while remaining conscious of the ethical considerations and societal impacts of AI technologies.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [46]:
topic = "NIS 2.0 in Germany"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on NIS 2.0 in Germany.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan Document

Topic: NIS 2.0 in Germany

Outline:
I. Introduction
- Brief overview of NIS 2.0 and its significance in Germany
- Mention of key players and latest trends in the field

II. Latest Trends in NIS 2.0 in Germany
- Increased focus on cybersecurity measures
- Emphasis on data protection and privacy regulations
- Integration of AI and machine learning technologies

III. Key Players in NIS 2.0 in Germany
- Federal Office for Information Security (BSI)
- Cybersecurity companies such as Siemens and 

In [47]:
Markdown(result)

# NIS 2.0 in Germany: Enhancing Cybersecurity Measures

In Germany, the National Information Security Strategy (NIS) 2.0 plays a crucial role in safeguarding critical infrastructure and digital assets against cyber threats. With the increasing reliance on digital technologies, the significance of NIS 2.0 cannot be overstated. Key players in this field, including the Federal Office for Information Security (BSI), are working diligently to address the latest trends and challenges in cybersecurity. The integration of AI and machine learning technologies has become a focal point in enhancing defense mechanisms and threat detection capabilities.

Recent trends in NIS 2.0 in Germany highlight an increased focus on cybersecurity measures, with organizations prioritizing data protection and privacy regulations. The evolving threat landscape necessitates proactive strategies to prevent cyber attacks and mitigate potential risks. The adoption of advanced technologies such as AI and machine learning enables organizations to strengthen their cybersecurity posture and respond effectively to emerging threats.

The Federal Office for Information Security (BSI) plays a pivotal role in shaping cybersecurity policies and implementing NIS 2.0 initiatives in Germany. Leading cybersecurity companies like Siemens and Deutsche Telekom contribute their expertise to enhance digital resilience and protect critical infrastructure. Collaboration with EU agencies and international partners ensures a coordinated approach to cybersecurity challenges and promotes information sharing among stakeholders.

Recent data breaches and cyber attacks underscore the importance of robust regulatory frameworks and compliance requirements in NIS 2.0 implementation. Organizations are leveraging success stories to showcase the effectiveness of cybersecurity measures and inspire others to adopt best practices. Stay informed about the latest developments in NIS 2.0 to stay ahead of evolving cyber threats and safeguard your digital assets effectively.

Cybersecurity professionals, IT experts, government officials, and business owners are the primary audience for NIS 2.0 content in Germany. By understanding the latest trends, key players, and regulatory updates, stakeholders can make informed decisions and implement proactive cybersecurity measures. Stay engaged with industry-specific resources and training programs to enhance your cybersecurity knowledge and protect your organization from cyber threats.

Stay informed about NIS 2.0 developments by following reputable cybersecurity news sources and industry publications. Promote cybersecurity training and awareness programs within your organization to build a culture of security and resilience. Implement NIS 2.0 measures effectively by leveraging resources and tools available to enhance your cybersecurity capabilities and protect against potential threats.

By incorporating SEO keywords such as NIS 2.0 Germany, cybersecurity trends, data protection regulations, cyber attacks prevention, and BSI updates, this blog post aims to provide valuable insights and promote awareness about NIS 2.0 in Germany. Empower yourself and your organization to stay ahead of cyber threats and enhance your cybersecurity posture in an increasingly digital world.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).